## Import data

As always, in dealing with real world data, the first step is to clean up the data. While the single family loan data published by Fannie Mae is relatively clean reading and combining all the files together are time-consuming. Ideally, I would like to read in all the acquisition file then combine them into a single file and do the same thing to performance data. However, the data (especially the performance data) is so huge that this is simply not doable on my computer. So I clean and combine them piece by piece. Below are the code for reading cand cleanning the data:

In [2]:
# import necessary libs

import seaborn as sns
import pandas as pd
import numpy as np
import os
import datetime as dt
%matplotlib inline
import matplotlib.pyplot as plt
low_memory=False

In [22]:
# setting up directory

DATA_DIR = '/Volumes/Backup Plus/Documents/Data Science/Projects/fannieMae_project/raw_data'
PROCESSED_DIR ='/Volumes/Backup Plus/Documents/Data Science/Projects/fannieMae_project/processed_data'

In [62]:
# list column names for the two datasets
Acquisition=[
           'LoanID',
           'Channel',
           'SellerName',
           'OrInterestRate',
           'OrUnpaidPrinc',
           'OrLoanTerm',
           'Origination',
           'FirstPayment',
           'OrLTV',
           'OrCLTV',
           'NumBorrowers',
           'DTIRat',
           'CreditScore',
           'FTHomeBuyer',
           'LoanPurpose',
           'PropertyType',
           'NumUnits',
           'OccType',
           'PropertyState',
           'Zip',
           'MortInsPerc',
           'ProductType',
           'CoCreditScore',
           'MortInsType',
           'RelocationMortgage'],
Performance=[
           'LoanID',
           'ReportPeriod',
           'Servicer',
           'CurrInterestRate',
           'CAUPB','LoanAge',
           'MonthsToMaturity',
           'AdMonthsToMaturity',
           'MaturityDate',
           'MSA',
           'CLDS',
           'ModFlag',
           'ZeroBalCode',
           'ZeroBalDate',
           'LastInstallDate',
           'ForeclosureDate',
           'DispositionDate', 
           'ForceclosureCost', 
           'PPRC',
           'AssetRecCost',
           'MHRC',
           'Tax',
           'NetSaleProceeds',
           'CreditEnhProceeds',
           'RPMWP',
           'OtherForceCloProcd',
           'NnonIntr_UPB',
           'PrinForgAmount',
           'RMWPFlag',
           'ForceWriteOffAmount',
           'ServicingIndicator']

In [4]:
# list the columns from Performance dataset to be kept

Performance_Select=[
        "LoanID",
        "MSA",
        "ForeclosureDate"
    ]

In [4]:
# define a function to quickly parse datetime columns

def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

In [6]:
# define concatenate_1 function to process Acquisition data, 
# drop rows with missing values, 
# append data from all year-quarter together

def concatenate_1(prefix="Acquisition"):
    files = os.listdir(DATA_DIR)
    full = []
    for f in files:
        if not f.startswith(prefix):
            continue
        data = pd.read_csv(os.path.join(DATA_DIR, f), sep="|", header=None, names=Acquisition, index_col=False)
        data = data[Acquisition]
        data = data.dropna()
        full.append(data)

    full = pd.concat(full, axis=0)

    full.to_csv(os.path.join(PROCESSED_DIR, "{}.txt".format(prefix)), sep="|", header=Acquisition, index=False)

In [7]:
# define concatenate_1 function to process Acquisition data, 
# drop rows with missing MSA and ForeclosureDate 
# append data from all year-quarter together

def concatenate_2(prefix="Performance"):
    files = os.listdir(DATA_DIR)
    full = []
    for f in files:
        if not f.startswith(prefix):
            continue
        data = pd.read_csv(os.path.join(DATA_DIR, f), sep="|", header=None, names=Performance, 
            dtype={'LoanID':'int64',
'ReportPeriod':'object',
'Servicer':'object',
'CurrInterestRate':'float64',
'CAUPB':'float64',
'LoanAge':'int64',
'MonthsToMaturity':'float64',
'AdMonthsToMaturity':'float64',
'MaturityDate':'object',
'MSA':'int64',
'CLDS':'object',
'ModFlag':'object',
'ZeroBalCode':'float64',
'ZeroBalDate':'object',
'LastInstallDate':'object',
'ForeclosureDate':'object',
'DispositionDate':'object',
'ForceclosureCost':'float64',
'PPRC':'float64',
'AssetRecCost':'float64',
'MHRC':'float64',
'Tax':'float64',
'NetSaleProceeds':'float64',
'CreditEnhProceeds':'float64',
'RPMWP':'float64',
'OtherForceCloProcd':'float64',
'NnonIntr_UPB':'float64',
'PrinForgAmount':'float64',
'RMWPFlag':'object',
'ForceWriteOffAmount':'float64',
'ServicingIndicator':'object'}
                           ,index_col=False)
        data = data[[
        "LoanID",
        "MSA",
        "ForeclosureDate"
    ]]
        data = data.dropna(subset=['MSA', 'ForeclosureDate'])
        full.append(data)

    full = pd.concat(full, axis=0)

    full.to_csv(os.path.join(PROCESSED_DIR, "{}.txt".format(prefix)), sep="|", header=[
        "LoanID",
        "MSA",
        "ForeclosureDate"
    ], index=False)

In [ ]:
# define concatenate_3 function to concatenate Acquisition and Performance data over all the years
def concatenate_3(prefix="Performance"):
    files = os.listdir(PROCESSED_DIR)
    full = []
    for f in files:
        if not f.startswith(prefix):
            continue
        data = pd.read_csv(os.path.join(PROCESSED_DIR, f), sep="|",index_col=False)
        full.append(data)
    full = pd.concat(full, axis=0)

    full.to_csv(os.path.join(PROCESSED_DIR, "{}.txt".format(prefix)), sep="|", index=False)

In [11]:
# use concatenate_1 function to process Acquisition data by year

if __name__ == "__main__":
    concatenate_1("Acquisition_2000")
    concatenate_1("Acquisition_2001")
    concatenate_1("Acquisition_2002")
    concatenate_1("Acquisition_2003")
    concatenate_1("Acquisition_2004")
    concatenate_1("Acquisition_2005")
    concatenate_1("Acquisition_2006")
    concatenate_1("Acquisition_2007")
    concatenate_1("Acquisition_2008")
    concatenate_1("Acquisition_2009")
    concatenate_1("Acquisition_2010")
    concatenate_1("Acquisition_2011")
    concatenate_1("Acquisition_2012")
    concatenate_1("Acquisition_2013")
    concatenate_1("Acquisition_2014")
    concatenate_1("Acquisition_2015")
    concatenate_1("Acquisition_2016")
    concatenate_1("Acquisition_2017")
    concatenate_1("Acquisition_2018")

In [9]:
# use concatenate_2 function to process Performance data by year

if __name__ == "__main__":
    concatenate_2("Performance_2000")
    concatenate_2("Performance_2001")
    concatenate_2("Performance_2002")
    concatenate_2("Performance_2003")
    concatenate_2("Performance_2004")
    concatenate_2("Performance_2005")
    concatenate_2("Performance_2006") 
    concatenate_2("Performance_2007")  
    concatenate_2("Performance_2008")
    concatenate_2("Performance_2009")
    concatenate_2("Performance_2010")
    concatenate_2("Performance_2011")
    concatenate_2("Performance_2012")
    concatenate_2("Performance_2013")
    concatenate_2("Performance_2014") 
    concatenate_2("Performance_2015") 
    concatenate_2("Performance_2016") 
    concatenate_2("Performance_2017")  
    concatenate_2("Performance_2018")

In [41]:
# use concatenate_3 to concatenate Acquisition and Performance data over all the years
concatenate_3("Performance")
concatenate_3("Acquisition")

## Clean data

That was a long way! Finally I was able to have concatenated Acquisition and Performance data over all the years available from Fannie Mae. Next steps are to:

* read the Acquisition data and Performance data
* Merge them by LoanID
* Derive new features
* Handle missing values

In [21]:
acqusition = pd.read_csv('/Volumes/Backup Plus/Documents/Data Science/Projects/fannieMae_project/processed_data/Acquisition.txt', sep="|",index_col=False)
performance = pd.read_csv('/Volumes/Backup Plus/Documents/Data Science/Projects/fannieMae_project/processed_data/Performance.txt', sep="|",index_col=False)
df=pd.merge(acqusition, performance, on='LoanID', how='left')

In [6]:
df['Default']=df['ForeclosureDate']
df['Default'].fillna(0, inplace=True)
df.loc[df['Default'] != 0, 'Default'] = 1
df['Default'] = df['Default'].astype(int)

In [7]:
df.rename(columns={'Origination':'loan_date',
                          'FirstPayment':'first_payment_date'}, 
                 inplace=True)

In [8]:
df[['loan_date', 'first_payment_date', 'ForeclosureDate']]=df[['loan_date', 'first_payment_date', 'ForeclosureDate']].apply(lookup)

In [9]:
df['first_payment_year']=df['first_payment_date'].dt.year
df['loan_year']=df['loan_date'].dt.year

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3194421 entries, 0 to 3194420
Data columns (total 30 columns):
LoanID                int64
Channel               object
SellerName            object
OrInterestRate        float64
OrUnpaidPrinc         int64
OrLoanTerm            int64
loan_date             datetime64[ns]
first_payment_date    datetime64[ns]
OrLTV                 float64
OrCLTV                float64
NumBorrowers          float64
DTIRat                float64
CreditScore           float64
FTHomeBuyer           object
LoanPurpose           object
PropertyType          object
NumUnits              int64
OccType               object
PropertyState         object
Zip                   int64
MortInsPerc           float64
ProductType           object
CoCreditScore         float64
MortInsType           float64
RelocationMortgage    object
MSA                   float64
ForeclosureDate       datetime64[ns]
Default               int64
first_payment_year    int64
loan_year          

## processing numeric and categorical columns seperately
* numeric columns: replace missing values with mean
* categorical columns: replace missing values with most frequent category

In [11]:
num_col_drop=["LoanID", "Zip", "MSA", 'first_payment_year', 'loan_year']
cat_col_add=['Zip', 'first_payment_year', 'loan_year']

In [12]:
# Create a boolean mask for numeric columns
num_mask = ((df.dtypes == int) | (df.dtypes == float))

# Get list of numeric column names
num_columns = df.columns[num_mask].tolist()
num=df[num_columns]

# drop "categorical" columns
num=num.drop(num_col_drop, axis=1)
num=num.fillna(num.mean())

## processing categorical columns

In [13]:
# Create a boolean mask for categorical columns
cat_mask = (df.dtypes == object)

# Get list of categorical column names
cat_columns = df.columns[cat_mask].tolist()

cat_columns.extend(cat_col_add)
cat=df[cat_columns]
cat=cat.astype('category')
cat=cat.fillna(cat.mode().iloc[0]) # fill na with most requent category

## concat num and cat columns

In [14]:
df_new = pd.concat([num, cat], axis=1).reset_index(drop=True)

## save the final data for EDA and ML later!

In [25]:
df_new.to_csv(os.path.join(PROCESSED_DIR, "Fannie_loans.csv"))